In [7]:
import os
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
import time

MAX_PIPES = 40  # For pipe1 through pipe39

In [8]:
class PipeNetwork:
    def __init__(self, max_pipes=MAX_PIPES):
        """
        Helper class to represent the pipe network structure.
        """
        self.max_pipes = max_pipes
        self.parent_map = {}  # Maps pipe_id -> parent_id
        self.child_map = {}   # Maps pipe_id -> list of child pipe_ids
        self.pipe_data = {}   # Maps pipe_id -> (length, radius, has_receiver)
        self.ancestors = {}   # Maps pipe_id -> list of ancestor pipe_ids
        self.descendants = {} # Maps pipe_id -> list of descendant pipe_ids
        self.distances = {}   # Maps (pipe_id1, pipe_id2) -> network distance
        
    def add_pipe(self, pipe_id, parent_id, length, radius, has_receiver):
        """Add a pipe to the network with its properties."""
        self.parent_map[pipe_id] = parent_id
        self.pipe_data[pipe_id] = (length, radius, has_receiver)
        
        # Update child_map
        if parent_id not in self.child_map and parent_id != -1:
            self.child_map[parent_id] = []
        if parent_id != -1:
            self.child_map[parent_id].append(pipe_id)
            
    def build_network_structure(self):
        """Build additional network structure information after all pipes are added."""
        # Calculate ancestors for each pipe
        for pipe_id in self.pipe_data.keys():
            self.ancestors[pipe_id] = self._get_ancestors(pipe_id)
            
        # Calculate descendants for each pipe
        for pipe_id in self.pipe_data.keys():
            self.descendants[pipe_id] = self._get_descendants(pipe_id)
            
        # Calculate network distances between all pairs of pipes
        pipe_ids = list(self.pipe_data.keys())
        for i in range(len(pipe_ids)):
            for j in range(i, len(pipe_ids)):
                pipe_id1 = pipe_ids[i]
                pipe_id2 = pipe_ids[j]
                distance = self._calculate_network_distance(pipe_id1, pipe_id2)
                self.distances[(pipe_id1, pipe_id2)] = distance
                self.distances[(pipe_id2, pipe_id1)] = distance
    
    def _get_ancestors(self, pipe_id):
        """Get all ancestors (parent, grandparent, etc.) of a pipe."""
        ancestors = []
        current = pipe_id
        while self.parent_map.get(current, -1) != -1:
            current = self.parent_map[current]
            ancestors.append(current)
        return ancestors
    
    def _get_descendants(self, pipe_id):
        """Get all descendants of a pipe recursively."""
        descendants = []
        if pipe_id in self.child_map:
            children = self.child_map[pipe_id]
            descendants.extend(children)
            for child in children:
                descendants.extend(self._get_descendants(child))
        return descendants
    
    def _calculate_network_distance(self, pipe_id1, pipe_id2):
        """
        Calculate the network distance (number of hops) between two pipes.
        """
        if pipe_id1 == pipe_id2:
            return 0
            
        # Check if pipe2 is an ancestor of pipe1
        if pipe_id2 in self.ancestors.get(pipe_id1, []):
            return len(self._get_path_to_ancestor(pipe_id1, pipe_id2))
            
        # Check if pipe1 is an ancestor of pipe2
        if pipe_id1 in self.ancestors.get(pipe_id2, []):
            return len(self._get_path_to_ancestor(pipe_id2, pipe_id1))
            
        # Find nearest common ancestor
        ancestors1 = self.ancestors.get(pipe_id1, [])
        ancestors2 = self.ancestors.get(pipe_id2, [])
        
        for ancestor in ancestors1:
            if ancestor in ancestors2:
                # Distance = path from pipe1 to ancestor + path from pipe2 to ancestor
                return (len(self._get_path_to_ancestor(pipe_id1, ancestor)) + 
                        len(self._get_path_to_ancestor(pipe_id2, ancestor)))
                
        # No common ancestor found (should not happen in a tree)
        return self.max_pipes
        
    def _get_path_to_ancestor(self, pipe_id, ancestor_id):
        """Get the path from a pipe to one of its ancestors."""
        path = []
        current = pipe_id
        while current != ancestor_id and self.parent_map.get(current, -1) != -1:
            path.append(current)
            current = self.parent_map[current]
        if current == ancestor_id:
            path.append(ancestor_id)
        return path
    
    def get_network_features(self, emitter_pipe_id=None):
        """
        Generate network-aware features for all pipes.
        
        Returns:
            numpy array of shape (max_pipes, feature_dim)
        """
        feature_dim = 10  # Adjust based on features below
        features = np.zeros((self.max_pipes, feature_dim), dtype=np.float32)
        
        for pipe_id, (length, radius, has_receiver) in self.pipe_data.items():
            if pipe_id >= self.max_pipes:
                continue
                
            # Base pipe features
            parent_id = self.parent_map.get(pipe_id, -1)
            
            # Network structural features
            num_ancestors = len(self.ancestors.get(pipe_id, []))
            num_descendants = len(self.descendants.get(pipe_id, []))
            
            # Distance to emitter if known
            distance_to_emitter = 0
            if emitter_pipe_id is not None:
                distance_to_emitter = self.distances.get((pipe_id, emitter_pipe_id), self.max_pipes)
            
            # Distance to root
            distance_to_root = len(self.ancestors.get(pipe_id, []))
            
            # Calculate shortest distance to a receiver
            min_distance_to_receiver = self.max_pipes
            for other_id, (_, _, other_has_receiver) in self.pipe_data.items():
                if other_has_receiver:
                    dist = self.distances.get((pipe_id, other_id), self.max_pipes)
                    min_distance_to_receiver = min(min_distance_to_receiver, dist)
            
            # Normalize features
            features[pipe_id-1] = [
                pipe_id / self.max_pipes,                  # Pipe ID (normalized)
                parent_id / self.max_pipes if parent_id > 0 else -0.1,  # Parent ID (normalized)
                length,                                    # Length
                radius,                                    # Radius
                1.0 if has_receiver else 0.0,              # Has receiver flag
                num_ancestors / self.max_pipes,            # Normalized number of ancestors
                num_descendants / self.max_pipes,          # Normalized number of descendants
                distance_to_root / self.max_pipes,         # Normalized distance to root
                distance_to_emitter / self.max_pipes,      # Normalized distance to emitter (0 during inference)
                min_distance_to_receiver / self.max_pipes  # Normalized distance to nearest receiver
            ]
            
        return features

In [9]:
class TreeAwareEmitterModel(nn.Module):
    def __init__(self, input_dim):
        super(TreeAwareEmitterModel, self).__init__()
        
        # Separate branches for network structure and receiver data
        self.network_encoder = nn.Sequential(
            nn.Linear(MAX_PIPES * 10, 512),  # 10 features per pipe
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU()
        )
        
        self.receiver_encoder = nn.Sequential(
            nn.Linear(MAX_PIPES * 10, 512),  # 10 features per receiver
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU()
        )
        
        # Combined layers
        self.combined_layers = nn.Sequential(
            nn.Linear(512, 256),  # 256 + 256 from both branches
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        
        # Separate heads for pipe classification and position regression
        self.pipe_head = nn.Linear(64, 1)  # Single value for pipe ID
        self.position_head = nn.Linear(64, 2)  # r, z coordinates
        
    def forward(self, x):
        # Split the input into network and receiver features
        network_features = x[:, :MAX_PIPES * 10]
        receiver_features = x[:, MAX_PIPES * 10:]
        
        # Process each branch
        network_encoding = self.network_encoder(network_features)
        receiver_encoding = self.receiver_encoder(receiver_features)
        
        # Combine branches
        combined = torch.cat((network_encoding, receiver_encoding), dim=1)
        features = self.combined_layers(combined)
        
        # Get predictions from each head
        pipe_pred = self.pipe_head(features)
        position_pred = self.position_head(features)
        
        # Combine predictions into single output
        return torch.cat((pipe_pred, position_pred), dim=1)

In [10]:
def predict_emitter_location(model_path, simulation_run_path):
    """
    Predict the emitter location for a single simulation run.
    
    Args:
        model_path: Path to the saved model (.pt file)
        simulation_run_path: Path to the simulation run folder
        
    Returns:
        dict with predicted emitter location
    """
    # Check if the model exists
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model not found at {model_path}")
        
    # Check if the simulation run exists
    if not os.path.exists(simulation_run_path):
        raise FileNotFoundError(f"Simulation run not found at {simulation_run_path}")
    
    # Setup device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Create pipe network
    network = PipeNetwork()
    receiver_data = {}
    
    # Process pipe and receiver data
    pipe_folders = sorted([
        f for f in os.listdir(simulation_run_path) 
        if os.path.isdir(os.path.join(simulation_run_path, f)) and f.startswith('pipe')
    ], key=lambda x: int(x.replace('pipe', '')))
    
    # First pass: collect pipe structure information
    for pipe_folder in pipe_folders:
        pipe_id = int(pipe_folder.replace('pipe', ''))
        if pipe_id > MAX_PIPES:
            continue
            
        pipe_path = os.path.join(simulation_run_path, pipe_folder)
        
        # Read pipe data
        sim_data_path = os.path.join(pipe_path, "simulation_data.txt")
        if os.path.exists(sim_data_path):
            with open(sim_data_path, 'r') as f:
                line = f.readline().strip().split()
                p_id = int(line[0])
                parent_id = int(line[1])
                length = float(line[2])
                radius = float(line[3])
                
                # Check for receivers
                has_receiver = len(glob.glob(os.path.join(pipe_path, "#*-Ring type.txt"))) > 0
                
                # Add pipe to network
                network.add_pipe(p_id, parent_id, length, radius, has_receiver)
        
        # Collect receiver data
        receiver_files = glob.glob(os.path.join(pipe_path, "#*-Ring type.txt"))
        for rec_file in receiver_files:
            with open(rec_file, 'r') as f:
                # Read receiver location and stats
                first_line = f.readline().strip().split()
                second_line = f.readline().strip().split(',')
                
                rec_pipe_id = int(first_line[0])
                r_coord = float(first_line[1])
                z_coord = float(first_line[2])
                
                # Parse statistics
                stats = [float(val.strip()) for val in second_line[:7]]  # Take up to 7 stats
                
                # Store receiver data
                if rec_pipe_id not in receiver_data:
                    receiver_data[rec_pipe_id] = []
                receiver_data[rec_pipe_id].append((r_coord, z_coord, stats))
    
    # Build network structure once all pipes are added
    network.build_network_structure()
    
    # Get enhanced network-aware features (without emitter info since we don't know it)
    network_features = network.get_network_features()
    
    # Process receiver data into a fixed-size feature vector
    receiver_features = np.zeros((MAX_PIPES, 10), dtype=np.float32)
    for pipe_id, receivers in receiver_data.items():
        if pipe_id >= MAX_PIPES:
            continue
            
        # Average stats if multiple receivers in the same pipe
        avg_r = np.mean([r for r, _, _ in receivers])
        avg_z = np.mean([z for _, z, _ in receivers])
        avg_stats = np.mean([s for _, _, s in receivers], axis=0)
        
        # Normalize pipe ID
        receiver_features[pipe_id-1] = np.concatenate([
            [pipe_id / MAX_PIPES, avg_r, avg_z],
            avg_stats if len(avg_stats) >= 7 else np.pad(avg_stats, (0, 7-len(avg_stats)))
        ])
    
    # Flatten and combine features
    flat_network_features = network_features.flatten()
    flat_receiver_features = receiver_features.flatten()
    
    # Create final feature vector
    combined_features = np.concatenate([flat_network_features, flat_receiver_features])
    
    # Convert to tensor
    features_tensor = torch.tensor(combined_features, dtype=torch.float32).unsqueeze(0).to(device)  # Add batch dimension
    
    # Load the model
    # Get input dimension from the features tensor
    input_dim = features_tensor.shape[1]
    model = TreeAwareEmitterModel(input_dim).to(device)
    
    # Load saved model weights
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    # Make prediction
    with torch.no_grad():
        prediction = model(features_tensor)
        
    # Format prediction
    pipe_id = int(round(prediction[0, 0].item() * MAX_PIPES))
    r_coord = prediction[0, 1].item()
    z_coord = prediction[0, 2].item()
    
    # Return results
    result = {
        'emitter_pipe_id': pipe_id,
        'emitter_r': r_coord,
        'emitter_z': z_coord
    }
    
    return result


In [17]:
# Example usage
if __name__ == "__main__":
    model_path = "best_emitter_model.pt"  # Path to your trained model
    
    # Path to the single simulation run you want to analyze
    simulation_run_path = "/Users/daghanerdonmez/Desktop/molecular-simulation-mlp/output-processing/Outputs_Copy/2025.06.16-17.52.09.869"  # Example path - replace with your run path
    
    try:
        result = predict_emitter_location(model_path, simulation_run_path)
        
        print("\n=== Emitter Location Prediction ===")
        print(f"Pipe ID: {result['emitter_pipe_id']}")
        print(f"R coordinate: {result['emitter_r']:.6f}")
        print(f"Z coordinate: {result['emitter_z']:.6f}")
        
        # If you have ground truth, you can compare
        ground_truth_path = os.path.join(simulation_run_path, "targetOutput.txt")
        if os.path.exists(ground_truth_path):
            with open(ground_truth_path, 'r') as f:
                truth = f.readline().strip().split()
                true_pipe = int(truth[0])
                true_r = float(truth[1]) 
                true_z = float(truth[2]) if len(truth) > 2 else 0.0
                
            print("\n=== Ground Truth ===")
            print(f"Pipe ID: {true_pipe}")
            print(f"R coordinate: {true_r:.6f}")
            print(f"Z coordinate: {true_z:.6f}")
            
            print("\n=== Accuracy ===")
            pipe_correct = result['emitter_pipe_id'] == true_pipe
            position_error = np.sqrt((result['emitter_r'] - true_r)**2 + (result['emitter_z'] - true_z)**2)
            
            print(f"Pipe prediction: {'CORRECT' if pipe_correct else 'INCORRECT'}")
            print(f"Position error: {position_error:.6f} units")
            
    except Exception as e:
        print(f"Error making prediction: {str(e)}")


=== Emitter Location Prediction ===
Pipe ID: 7
R coordinate: 0.000456
Z coordinate: -0.003102

=== Ground Truth ===
Pipe ID: 33
R coordinate: 0.000406
Z coordinate: 0.002212

=== Accuracy ===
Pipe prediction: INCORRECT
Position error: 0.005314 units
